In [1]:
import os

from google.colab import drive
drive.mount('/content/drive')

os.getcwd()



os.chdir('/content/drive/MyDrive/deep4')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install xarray

!pip install captum

!pip install cartopy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [8]:
!pip install --upgrade xarray torch captum

!pip install xarray[complete]

import xarray as xr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.4 MB/s eta 0:00:00
  Attempting uninstall: xarray
    Found existing installation: xarray 2025.1.2
    Uninstalling xarray-2025.1.2:
      Successfully uninstalled xarray-2025.1.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.0 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
^C


In [4]:
import xarray as xr

In [2]:
import xarray as xr



import torch
import captum
from torch.utils.data import DataLoader, random_split

!pip install xskillscore

In [3]:
import xarray as xr
import torch
from torch.utils.data import DataLoader, random_split
import deep4downscaling.viz
import deep4downscaling.trans
import deep4downscaling.deep.loss
import deep4downscaling.deep.physics
import deep4downscaling.deep.PhysicsData
import deep4downscaling.deep.utils
import deep4downscaling.deep.models
import deep4downscaling.deep.phystrain
import deep4downscaling.deep.pred
import deep4downscaling.metrics
import deep4downscaling.metrics_ccs
import numpy as np


In [4]:
# --- Load predictor and predictand ---
predictor_filename = 'pred_era5.nc'
predictor = xr.open_dataset(predictor_filename)

predictand_filename = 'tmin_25km.nc'
predictand = xr.open_dataset(predictand_filename)


In [5]:
# --- Optionally load P/E data ---
p_filename = 'tmin_25km.nc'
e_filename = 'tmax_25km.nc'
p_da = xr.open_dataset(p_filename)
e_da = xr.open_dataset(e_filename)


In [6]:
# --- Define training/test periods ---
years_train = ('1983', '2010')
years_test = ('2011', '2015')

predictor = predictor.rename({'valid_time': 'time'}).squeeze(drop=True)

x_train = predictor.sel(time=slice(*years_train))
y_train = predictand.sel(time=slice(*years_train))

# For physics, we select the same period for P and E:
p_train = p_da.sel(time=slice(*years_train))
e_train = e_da.sel(time=slice(*years_train))

x_test = predictor.sel(time=slice(*years_test))
y_test = predictand.sel(time=slice(*years_test))


In [7]:
# --- Preprocess / Standardize the data ---
x_train_stand = deep4downscaling.trans.standardize(data_ref=x_train, data=x_train)
y_mask = deep4downscaling.trans.compute_valid_mask(y_train)

# Stack grid points if needed
y_train_stack = y_train.stack(gridpoint=('lat', 'lon'))
y_mask_stack = y_mask.stack(gridpoint=('lat', 'lon'))
y_mask_stack_filt = y_mask_stack.where(y_mask_stack==1, drop=False)
y_train_stack_filt = y_train_stack.where(
    y_train_stack['gridpoint'] == y_mask_stack_filt['gridpoint'], drop=False
)


In [8]:
# Convert x and y to numpy arrays
x_train_stand_arr = deep4downscaling.trans.xarray_to_numpy(x_train_stand)
y_train_arr = deep4downscaling.trans.xarray_to_numpy(y_train_stack_filt)


In [9]:
# Also convert the physics variables (make sure they are in the right shape)
p_train_arr = deep4downscaling.trans.xarray_to_numpy(p_train)
e_train_arr = deep4downscaling.trans.xarray_to_numpy(e_train)


In [10]:
p_train_arr.shape

(10227, 100, 180)

In [11]:
e_train_arr.shape

(10227, 100, 180)

In [12]:
y_train_arr.shape

(10227, 18000)

In [13]:
predictand.Tmin

<xarray.DataArray 'Tmin' (time: 12419, lat: 100, lon: 180)> Size: 894MB
[223542000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 99kB 1983-01-01 1983-01-02 ... 2016-12-31
  * lon      (lon) float64 1kB -19.88 -19.62 -19.38 -19.12 ... 24.38 24.62 24.88
  * lat      (lat) float64 800B 0.125 0.375 0.625 0.875 ... 24.38 24.62 24.88
Attributes:
    standard_name:       Tmin
    long_name:           Climate Hazards Center Tmin
    units:               degree Celcius
    time_step:           daily
    geostatial_lat_min:  -40.0
    geostatial_lat_max:  40.0
    geostatial_lon_min:  -20.0
    geostatial_lon_max:  55.0

In [14]:
# --- Build the Dataset and DataLoaders ---
from torch.utils.data import random_split

from deep4downscaling.deep.PhysicsData import PhysicsDataset
dataset = PhysicsDataset(x_train_stand_arr, y_train_arr, p_train_arr, e_train_arr)


In [15]:
# Split dataset into training and validation sets (here a 90/10 split)
train_dataset, valid_dataset = random_split(dataset, [int(0.9 * len(dataset)), len(dataset) - int(0.9 * len(dataset))])

batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)


In [16]:
# --- Build the model ---
model_name = 'cnn_tmin_with_physics'
model = deep4downscaling.deep.models.DeepESDtas(
    x_shape=x_train_stand_arr.shape,
    y_shape=y_train_arr.shape,  # Note: y_shape is for the main predictand (e.g., temperature)
    filters_last_conv=1,
    stochastic=False
)


In [17]:
model

DeepESDtas(
  (conv_1): Conv2d(15, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_2): Conv2d(50, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_3): Conv2d(25, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (out): Linear(in_features=496, out_features=18000, bias=True)
)

In [18]:
# --- Define the physics informed loss ---
from deep4downscaling.deep.physics import PhysicsMseLoss
# You can adjust lambda_phys as needed.
loss_function = PhysicsMseLoss(ignore_nans=True, lambda_phys=0)




In [19]:
num_epochs = 10000
patience_early_stopping = 20

learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(),
                             lr=learning_rate)

model

device = ('cuda' if torch.cuda.is_available() else 'cpu')

MODELS_PATH = '/content/drive/MyDrive/deep4/notebooks/models'
DATA_PATH = './data/input'
FIGURES_PATH = './figures'


In [20]:
model

DeepESDtas(
  (conv_1): Conv2d(15, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_2): Conv2d(50, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_3): Conv2d(25, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (out): Linear(in_features=496, out_features=18000, bias=True)
)

In [21]:

# --- Train the Model using the Standard Training Loop ---
train_loss, val_loss = deep4downscaling.deep.phystrain.standard_training_loop(
    model=model,
    model_name=model_name,
    model_path=MODELS_PATH,  # ensure that MODELS_PATH is defined in your context
    device='cuda',
    num_epochs=num_epochs,
    loss_function=loss_function,
    optimizer=optimizer,
    train_data=train_dataloader,
    valid_data=valid_dataloader,
    patience_early_stopping=patience_early_stopping
)


Epoch 1 (3.63 secs) | Training Loss 253.2832 | Validation Loss 27.4008 (Model saved)
Epoch 2 (3.04 secs) | Training Loss 20.5941 | Validation Loss 17.2768 (Model saved)
Epoch 3 (3.06 secs) | Training Loss 13.7926 | Validation Loss 12.0841 (Model saved)
Epoch 4 (3.05 secs) | Training Loss 10.0444 | Validation Loss 9.2345 (Model saved)
Epoch 5 (3.08 secs) | Training Loss 7.9709 | Validation Loss 7.593 (Model saved)
Epoch 6 (3.05 secs) | Training Loss 6.8249 | Validation Loss 6.6826 (Model saved)
Epoch 7 (3.04 secs) | Training Loss 6.0428 | Validation Loss 5.9735 (Model saved)
Epoch 8 (3.04 secs) | Training Loss 5.4489 | Validation Loss 5.328 (Model saved)
Epoch 9 (3.07 secs) | Training Loss 4.9858 | Validation Loss 5.0481 (Model saved)
Epoch 10 (3.03 secs) | Training Loss 4.6003 | Validation Loss 4.498 (Model saved)
Epoch 11 (3.02 secs) | Training Loss 4.2883 | Validation Loss 4.2391 (Model saved)
Epoch 12 (3.05 secs) | Training Loss 4.053 | Validation Loss 3.9626 (Model saved)
Epoch 13 

In [22]:
# --- After training, load best weights and compute predictions ---
model.load_state_dict(torch.load(f'{MODELS_PATH}/{model_name}.pt'))

x_test_stand = deep4downscaling.trans.standardize(data_ref=x_train, data=x_test)
pred_test = deep4downscaling.deep.pred.compute_preds_standard(
    x_data=x_test_stand,
    model=model,
    device=device,
    var_target='Tmin',
    mask=y_mask,
    batch_size=16
)


<ipython-input-22-02decda19090>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'{MODELS_PATH}/{model_name}.pt'))


In [23]:
pred_test.to_netcdf('cnn1_tmin_with_physics.nc')
